In [1]:
import glob
import math
import os
import random
import shutil
from pathlib import Path

import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
# for training and testing

class TrainImgLabels(Dataset):
    def __init__(self, img_file, batch_size=16, augment=False, class_weights=False):
        with open(img_file, "r") as fin:
            self.img_paths = list(fin.read().splitlines())
        n = len(self.img_paths)
        batch_idx = np.floor(np.arrange(n) / batch_size).astype(np.int)
        num_batch = baatch_idx[-1] + 1
        
        self.n = n
        self.batch = batch_idx
        self.augment = augment
        self.class_weights = class_weights
        
        self.label_files = [x.replace('image', 'bbox').               #################what is this for#######
                        replace('.jpeg', '.txt').
                        replace('.jpg', '.txt').
                        replace('.bmp', '.txt').
                        replace('.png', '.txt') for x in self.img_paths] 
        
        # preload labels
        self.imgs = [None] * n
        self.labels = [np.zeros((0, 5))] * n
        iter = tqdm(self.label_files, desc='Reading labels')
        for i, file in enumerate(iter):
            try:
                with open(file, 'r') as f:
                    l = np.array([x.split() for x in f.read().splitlines()], dtype=np.float32)
                    if l.shape[0]:
                        assert l.shape[1] == 5, '> 5 label columns: %s' % file
                        assert (l >= 0).all(), 'negative labels: %s' % file
                        assert (l[:, 1:] <= 1).all(), 'non-normalized or out of bounds coordinate labels: %s' % file
                        self.labels[i] = l
            except FileNotFoundError:
                pass  # print('Warning: missing labels for %s' % self.img_files[i])  # missing label file
        assert len(np.concatenate(self.labels, 0)) > 0, 'No labels found. Incorrect label paths provided.'
        
        
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        if self.class_weights:
            index = self.indices[index] ########################what is this for?##################
        
        # images
        img_path = self.img_paths[index]
        img = cv2.imread(img_path)
        assert img is not None, "Image not found" + img_path
        
        # load labels
        label = self.labels[index]
        
        

In [ ]:
# for inference

In [ ]:
# for web cam